In [ ]:
!pip install pandas
!pip install "altair[all]"

In [58]:
import pandas as pd
import altair as alt

In [59]:
df = pd.read_csv("../data/pudding_love_songs_2023.csv")
df.head()

,song_title,pipe_delimited_artist_list,love_song_category,top_10_debut_date_as_decimal,total_weeks_in_top_10
0,Yakety Yak,The Coasters,NaN,1958.59,1
1,Splish Splash,Bobby Darin,NaN,1958.59,3
2,When,Kalin Twins,Serenade,1958.59,5
3,Hard Headed Woman,Elvis Presley|The Jordanaires,It's Complicated,1958.59,2
4,Patricia,Perez Prado,NaN,1958.59,6


In [60]:
# Preprocess the data by replacing NaN with Uncategorized
df["love_song_category"] = df["love_song_category"].fillna("Uncategorized")
df.head()

,song_title,pipe_delimited_artist_list,love_song_category,top_10_debut_date_as_decimal,total_weeks_in_top_10
0,Yakety Yak,The Coasters,Uncategorized,1958.59,1
1,Splish Splash,Bobby Darin,Uncategorized,1958.59,3
2,When,Kalin Twins,Serenade,1958.59,5
3,Hard Headed Woman,Elvis Presley|The Jordanaires,It's Complicated,1958.59,2
4,Patricia,Perez Prado,Uncategorized,1958.59,6


In [61]:
# Count the number of songs that belong to each category and sorted in a decreasing order
num_categories = df["love_song_category"].unique()
count_per_category = df.groupby('love_song_category')['song_title'].count().sort_values(ascending=False).reset_index(name='count')

# Add explanation for each song category (adapted from https://pudding.cool/2024/11/love-songs/)
explanation = {
    "Serenade": "Unmistakably about romantic love and devotion, sung from one person to another",
    "Heartache": "But what happens if you love them, but they just... don’t? Maybe you broke up, or maybe it’s just unrequited",
    "It's Complicated": "The messier side of romance. What about when a relationship isn’t clearly good or bad? Maybe you fight constantly. Maybe they’re unfaithful. But you still try to make it work",
    "Pursuit": "When you love someone, and it might become something more? You just spotted someone, your heart is beating fast, and who KNOWS where this thing might lead?",
    "Sexual Confidence": "Songs that get a little steamy? Think artists like Nicki Minaj and Drake, who dominate this category that's all about getting into bed with someone.",
    "Good Riddance": 'The relationship is clearly over, but the songwriter’s heartbreak has resurrected into... righteous power? They’re not crying, they’re yelling “Good Riddance!”',
    "Love Song for the Self": "Women-pioneered Love Song for the Self is the most modern of all the genres. And while it’s not written to a lover, it’s often written in reaction to romantic rejection.",
    "Uncategorized": "Songs that don't fit anywhere else"
}

# Merge the explanation to the count dataset
explanation_df = pd.DataFrame.from_dict(explanation, orient="index").reset_index()
explanation_df.columns = ["love_song_category", "explanation"]

In [62]:
count_per_category

,love_song_category,count
0,Uncategorized,1798
1,Serenade,1042
2,Heartache,827
3,It's Complicated,484
4,Pursuit,445
5,Sexual Confidence,315
6,Good Riddance,125
7,Love Song for the Self,105


In [63]:
explanation_df

,love_song_category,explanation
0,Serenade,"Unmistakably about romantic love and devotion,..."
1,Heartache,"But what happens if you love them, but they ju..."
2,It's Complicated,The messier side of romance. What about when a...
3,Pursuit,"When you love someone, and it might become som..."
4,Sexual Confidence,Songs that get a little steamy? Think artists ...
5,Good Riddance,"The relationship is clearly over, but the song..."
6,Love Song for the Self,Women-pioneered Love Song for the Self is the ...
7,Uncategorized,Songs that don't fit anywhere else


In [64]:
merged_df = pd.merge(count_per_category, explanation_df, on="love_song_category")

In [65]:
merged_df

,love_song_category,count,explanation
0,Uncategorized,1798,Songs that don't fit anywhere else
1,Serenade,1042,"Unmistakably about romantic love and devotion,..."
2,Heartache,827,"But what happens if you love them, but they ju..."
3,It's Complicated,484,The messier side of romance. What about when a...
4,Pursuit,445,"When you love someone, and it might become som..."
5,Sexual Confidence,315,Songs that get a little steamy? Think artists ...
6,Good Riddance,125,"The relationship is clearly over, but the song..."
7,Love Song for the Self,105,Women-pioneered Love Song for the Self is the ...


In [67]:
select = alt.selection_point(name="select", on="click")
highlight = alt.selection_point(name="highlight", on="pointerover", empty=False)

stroke_width = (
    alt.when(select).then(alt.value(2, empty=False))
    .when(highlight).then(alt.value(1))
    .otherwise(alt.value(0))
)

chart = alt.Chart(merged_df).mark_bar(
    fill="#E78483", stroke="black", cursor="pointer"
).encode(
    alt.X('count').title("Number of Love Songs"),
    alt.Y("love_song_category").title("Category").sort('-x'),
    fillOpacity=alt.when(select).then(alt.value(1)).otherwise(alt.value(0.3)),
    strokeWidth=stroke_width,
).properties(
    width=800,
    height=300
).configure_scale(bandPaddingInner=0.2).add_params(select, highlight)

chart

alt.Chart(...)